In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from matplotlib.colors import LogNorm    
import RANSAC
rcParams['font.family'] = 'serif'

%matplotlib inline

# Plane fit

Se pretende determinar los parametros $A,B$ y $C$ de la ecuación del plano $Ax+By+Cz = 0$ que representa un plano dado un conjunto de puntos $\{[x_{1},y_{1},z_{1}]...[x_{n},y_{n},z_{n}]\}$ con el método de mínimos cuadrados combinado con RANSAC. La función a minimizar será la suma de las distancias de un punto con el plano.

Como método de visualizar el ajuste se implementa una función cuyo propósito es rotar los puntos de tal manera que el vector normal sea $[0,0,1]$, es decir, quede perpendicular al plano $xy$. Luego si el ajuste es óptimo, en el plot $xz$ e $yz$ los puntos debería aparecer alineados de modo que se aprecie que el disco está de visto "edge-on" y en el plot $xy$ se debería ver "face-on".

In [ ]:
def rotar_plano(A,B,C,points):
    norm_vec = np.matrix([A,B,C])

    alpha0 = -( np.pi/2 - np.arcsin(C/np.linalg.norm(norm_vec)))
    if B>0:
        phi0 = -np.arccos(A/np.linalg.norm([A,B])) 
    else:
        phi0 = np.arccos(A/np.linalg.norm([A,B]))
        
    # Matriz de rotacion en torno al eje z
    rot_z = np.matrix([[np.cos(phi0),-np.sin(phi0),0],[np.sin(phi0),np.cos(phi0),0],[0,0,1]])
    # Matriz de rotacion en torno al eje Y
    rot_y = np.matrix([[np.cos(alpha0),0,np.sin(alpha0)],[0,1,0],[-np.sin(alpha0),0,np.cos(alpha0)]])
    rot_total = (rot_y*rot_z)
    rot_points = (rot_total*points.T).A.T
    rotA,rotB,rotC = (rot_total*norm_vec.T).A.T[0]
    
    return rotA,rotB,rotC,rot_points,rot_total

def plot_sim(fig,ax,points,title,mod='points',a=[1,1,1],c=None,bins=None,recta=False,VectN=None,x_array=None):
    
    fig.suptitle(title, fontsize=20)
    
    if mod == "histogram":
        ax01= ax[0].hist2d(points[:,0],points[:,1], bins=bins,norm=LogNorm())
        ax02= ax[1].hist2d(points[:,0],points[:,2], bins=bins,norm=LogNorm())
        ax03= ax[2].hist2d(points[:,1],points[:,2], bins=bins,norm=LogNorm())
    elif mod == 'points':
        ax[0].plot(points[:,0],points[:,1],c[0],ms=1,alpha=a[0])
        ax[1].plot(points[:,0],points[:,2],c[1],ms=1,alpha=a[1])
        ax[2].plot(points[:,1],points[:,2],c[2],ms=1,alpha=a[2])
        
    if recta:
        ax[1].plot(x_array,-(VectN[0]*x_array)/VectN[2],'-k')
        ax[2].plot(x_array,-(VectN[1]*x_array)/VectN[2],'-k')

        
    ax[0].set(xlabel='x', ylabel='y')
    ax[0].minorticks_on()
    
    ax[1].set(xlabel='x', ylabel='z')
    ax[1].minorticks_on()

    ax[2].set(xlabel='y', ylabel='z')
    ax[2].minorticks_on()

## Fichero

En esta parte se leen las carpetas que contienen los .dat, se extráen los archivos correspondientes a el disco primario y circumbinario. Luego se aplica la función para extraer la ecuaciones de los planos a cada archivo y es guardado en un archivo de texto donde los primeros tres números de cada fila corresponden a los parámetros $A, B, C$ del plano circumbinario y los últimos tres corresponden a los parámetros del disco primario. Se crearán tantanas filas como archivos .dat haya.

In [ ]:
import glob

aux = ['ecc000','ecc025','ecc075','REF']

for element in aux:
    simulacion = element

    name_files_primary = glob.glob('/home/pedro/Proyecto_2/'+simulacion+'/xyzprimary*')
    name_files_primary.sort()

    name_files_circumbinary = glob.glob('/home/pedro/Proyecto_2/'+simulacion+'/xyzcircumbinary*')
    name_files_circumbinary.sort()

    filename = simulacion + '_plane_params.csv'
    fil=open(filename,'w')

    for i in range(len(name_files_primary)):
        pfile=np.loadtxt(name_files_circumbinary[i])
        pfile =np.array(pfile)
        x_value = pfile[:,0]
        y_value = pfile[:,1]
        z_value = pfile[:,2]
        circumbinary_points = np.stack([x_value,y_value,z_value],axis=1)

        pfile=np.loadtxt(name_files_primary[i])
        pfile =np.array(pfile)
        x_value = pfile[:,0]
        y_value = pfile[:,1]
        z_value = pfile[:,2]
        primary_points = np.stack([x_value,y_value,z_value],axis=1)

        _,_,CvectN,_ = RANSAC.RANSAC_N(circumbinary_points,1000,3,1,80)
        _,_,PvectN,_ = RANSAC.RANSAC_N(primary_points,1000,3,1,80)

        print 'Simulación:',simulacion
        print 'Vector normal plano circumbinario: [{0:1.4f}'.format(CvectN[0]),'{0:1.4f}'.format(CvectN[1]),'{0:1.4f}]'.format(CvectN[2])
        print 'Vector normal plano primario: [{0:1.4f}'.format(PvectN[0]),'{0:1.4f}'.format(PvectN[1]),'{0:1.4f}]'.format(PvectN[2])
        print '\n'

        fil.write(str(CvectN[0])+' '+str(CvectN[1])+' '+str(CvectN[2])+' '+
                  str(PvectN[0])+' '+str(PvectN[1])+' '+str(PvectN[2])+'\n')

    fil.close()

En esta última parte se leen los archivos generados y se grafica la evolución del ańgulo entre los vectores normales.


In [ ]:
aux = ['ecc000','ecc025','REF','ecc075']

plt.figure()
for element in aux:
    pfile=np.loadtxt('./'+element+'_plane_params.csv')
    pfile =np.array(pfile)
    CvectN = np.stack(np.array([pfile[:,0],pfile[:,1],pfile[:,2]]),axis=1)
    PvectN = np.stack(np.array([pfile[:,3],pfile[:,4],pfile[:,5]]),axis=1)
    cos_angle = CvectN[:,0]*PvectN[:,0] + CvectN[:,1]*PvectN[:,1] + CvectN[:,2]*PvectN[:,2]
    angle = np.degrees(np.arccos(cos_angle))
    num = [50*(i+1) for i in range(6)]
    
    plt.plot(num,angle,'o-',label=element)
    
plt.xlabel('time',size=10)
plt.ylabel('angle [degrees]',size=10)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.minorticks_on()
